# S2S Zonal statistics

Zonal statistics are run on the standardized [H3 grid](https://h3geo.org/docs/core-library/restable/); the process is run on a country-by-country basis.

For the zonal statistics, each zonal statistic is run against the source dataset as a whole, then it is stratified by urban classification from the European Commission - [GHS-SMOD](https://ghsl.jrc.ec.europa.eu/ghs_smod2019.php). This creates an summary dataset that has the standard zonal stats columns (SUM, MEAN, MAX, MIN) as well as the same for urban areas (SUM_urban, MEAN_urban, MAX_urban, MIN_urban).

In [1]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
#import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.append("../src")
import h3_helper
import country_zonal

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
h3_level = 6

admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
out_folder = f"/home/wb411133/projects/Space2Stats/Population"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
global_urban = "/home/public/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"

## Run analysis on population by gender and age

In [3]:
population_folder = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/GLOBAL_1km_Demographics"
pop_files = [os.path.join(population_folder, x) for x in os.listdir(population_folder) if x.endswith("1km.tif")]

In [4]:
# get a list of h3 levels to process
h3_0_list = h3_helper.generate_lvl0_lists(6, return_gdf=True, buffer0=False)

In [5]:
AWS_S3_BUCKET = 'wbg-geography01'
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

def run_zonal(gdf, cur_raster_file, out_file):
    cName = f'{os.path.basename(os.path.dirname(out_file))}-{os.path.basename(cur_raster_file)}'
    tPrint(f'Starting {cName}')
    res = rMisc.zonalStats(gdf, cur_raster_file, minVal=0)
    res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
    res['id'] = gdf['shape_id'].values
    res.to_csv(
        f"s3://{AWS_S3_BUCKET}/{out_file}",
        index=False,
        storage_options={
            "key": AWS_ACCESS_KEY_ID,
            "secret": AWS_SECRET_ACCESS_KEY,
            "token": AWS_SESSION_TOKEN,
        },
    )
    #res.to_csv(out_file)
    tPrint(f'**** finished {cName}')
    return(res)
    

In [ ]:

# set up mp arguments
arg_list = []
processed_list = []
keep_processing = True
while keep_processing:
    arg_list = []
    processed_list = []
    for h3_0_key, cur_gdf in h3_0_list.items():
        for pop_file in pop_files:
            filename = os.path.basename(f'{pop_file.replace(".tif", "")}_zonal.csv')
            out_s3_key = f'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/{h3_0_key}/{filename}'
            full_path = os.path.join("s3://", AWS_S3_BUCKET, out_s3_key)        
            try:
                tempPD = pd.read_csv(full_path)
                processed_list.append(filename)
            except:
                arg_list.append([cur_gdf, pop_file, out_s3_key])
    keep_processing = len(arg_list) != 0
    tPrint(f'Remaining: {len(arg_list)}\t Processed: {len(processed_list)}')


12:00:41	Remaining: 2162	 Processed: 2230
12:27:57	Remaining: 1897	 Processed: 2495


In [ ]:
for args in arg_list:
    run_zonal(*args)

In [ ]:
with multiprocessing.Pool(processes=min([70,len(arg_list)])) as pool:
    results = pool.starmap(run_zonal, arg_list[:5])

In [ ]:
rMisc.zonalStats?

In [ ]:
arg_list[0][0]